In [1]:
import sys, os
from pathlib import Path
import time

parent_directory = Path.cwd().parent
# Add the parent directory to sys.path
# print(parent_directory)
sys.path.append(str(parent_directory))
# print(sys.path)

In [2]:
from modules.rds_access import execute_sql_query, DatabaseConnection
import os
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:

conn_params = {                                                   
        "host": os.environ.get("POSTGRE_HOST", ''),                                
        "port": 5432,                                                              
        "user": os.environ.get("POSTGRE_USER", ''),                                
        "password": os.environ.get("POSTGRE_PWD", '')                              
    }        

In [4]:
conn_params['database'] = "kugou"

sql_query = """SELECT audio_id, singer_id, ks.language, ori_author_name, 
work_name, ks.cid, ks.scrape_date, ks.combine_count as comment_count, 
'NA' as likes_count, 'NA' as stream_count_1, 'NA' as stream_count_2,
ks.album_id, ks.publish_company, 
ks.raw_json_1, singer_ids, singer_names, combine_count_v2, 
album_name, ka.album_name
FROM kugou_songs ks
LEFT JOIN kugou_albums ka ON ks.album_id = ka.album_id
WHERE work_name ilike '%SuperFX%'
"""
with DatabaseConnection(conn_params) as conn:
    df = execute_sql_query(sql_query, conn)
    print(df.shape)

print(', '.join(df.columns))    
df.head().T

(12, 19)
audio_id, singer_id, language, ori_author_name, work_name, cid, scrape_date, comment_count, likes_count, stream_count_1, stream_count_2, album_id, publish_company, raw_json_1, singer_ids, singer_names, combine_count_v2, album_name, album_name


,0,1,2,3,4
audio_id,48408857,291406516,78767123,78767208,78767209
singer_id,None,None,None,None,None
language,None,英语,None,None,None
ori_author_name,Siriusmo,WesHxc,Thomas Bergersen,Thomas Bergersen,Thomas Bergersen
work_name,SUPERFX,SuperFX,SuperFX(Hit)Orchestra Hit I,SuperFX(Hit)Orchestra Hit III,SuperFX(Hit)Orchestra Hit VIII
cid,65216314,327163370,109253321,109253353,109253312
scrape_date,2024-03-16 19:00:31.410000+00:00,2024-03-17 08:48:12.167000+00:00,2024-03-16 21:43:45.347000+00:00,2024-03-16 21:43:45.377000+00:00,2024-03-16 21:43:45.378000+00:00
comment_count,0,0,1,1,1
likes_count,NA,NA,NA,NA,NA
stream_count_1,NA,NA,NA,NA,NA


In [5]:
# function to filter the corresponding lines
def row_filter(row):
    return row['ori_author_name'].lower().strip() == 'nana' and 'lonely' in row['work_name'].lower()
    

In [6]:
df_filtered = df.loc[df.apply(row_filter, axis=1)]

In [7]:
df_filtered.to_csv('result_filtered.csv')

In [11]:
start = time.time()
# query for the netease_max database. 

# sql_query = """SELECT album_id, song_id, song_name, artist_id, copyright_id, deprecated_artist_name, artist_ids,
# comment_count, audit_songs.artist_array, album_name, release_date, company, song_array
# FROM audit_songs
# INNER JOIN audit_albums USING (album_id)
# where '102714' = any(audit_songs.artist_array)
# or '792429' = any(audit_songs.artist_array)
# or '45336' = any(audit_songs.artist_array)

# """
sql_query = """
SELECT audit_songs.album_id, audit_songs.song_id, audit_songs.song_name, audit_songs.artist_id, 
copyright_id, deprecated_artist_name, artist_ids,
audit_songs.comment_count, audit_songs.artist_array, album_name, release_date, 
company, song_array, likes as likes_count, 
award.play_num as award_play_num, award.comment_count as award_comment_count, 
top_songs.play_count as top_songs_play_count
FROM audit_songs
INNER JOIN audit_albums on audit_songs.album_id = audit_albums.album_id
INNER JOIN honor_single_award as award on award.song_id = audit_songs.song_id
INNER JOIN honor_top_songs as top_songs on audit_songs.song_id::text = top_songs.song_id
where audit_songs.song_id = '1909555981' or audit_songs.song_id = '2032918401'

"""

conn_params['database'] = "netease_max"
with DatabaseConnection(conn_params) as conn:
    df = execute_sql_query(sql_query, conn)
#     print(sql_query)
df.head(100).T

,0,1,2
album_id,162375136,138432809,138432809
song_id,2032918401,1909555981,1909555981
song_name,where is the love?,where is the love?,where is the love?
artist_id,287416,40684,40684
copyright_id,743010,743010,743010
deprecated_artist_name,"Ni/Co's Covers,Ni/Co","Ni/Co,Ni/Co's Covers","Ni/Co,Ni/Co's Covers"
artist_ids,"51091605,27842648","27842648,51091605","27842648,51091605"
comment_count,104,1329,1329
artist_array,"[51091605, 27842648]","[27842648, 51091605]","[27842648, 51091605]"
album_name,where is the love?,Songs We Wish We Wrote: Y2K Edition,Songs We Wish We Wrote: Y2K Edition


In [41]:
start = time.time()
conn_params['database'] = "qqmusicv2"
# sql_query = """SELECT song_mid, song_name, album_mid, album_name, artist_mid, singer_names,
# comment_number, singer_mids, company, release_date
# FROM songs
# WHERE '002Vj8o12l10eK' = ANY(singer_mids)
# or '001fbZH44WRuKW' = ANY(singer_mids)
# or '000Eqw823oivl5' = ANY(singer_mids)

# """
sql_query = """SELECT song_mid, song_name, album_mid, album_name, artist_mid, singer_names,
comment_number as comment_count, singer_mids, company, release_date, 
'NA' as likes_count, 'NA' as stream_count_1, 'NA' as stream_count_2
FROM songs
WHERE song_name ilike '%SuperFX%'
"""
# sql_query = "select * from songs limit 2"


with DatabaseConnection(conn_params) as conn:
    df = execute_sql_query(sql_query, conn)
    print(df.shape)
    
end = time.time()
print("time elapsed is: ", end - start)
df.head().T

(48, 13)
time elapsed is:  3.328744888305664


,0,1,2,3,4
song_mid,003JgrFD2Pf11c,004ZahSf3B7B0a,000BRYMu412j4S,003948Ce30cw0W,0045qdCt1weXDz
song_name,SuperFX (Static + Rise) - Alien Encounter (Alien Encounter),SUPERFX,SuperFX (Hit + Rise) - Ignition,SuperFX (Hit + Static) - System Failure,SuperFX (Static + Pounding) - Toxic Air
album_mid,,003nNgd80yiqqa,003w8wlS3RYIbM,003w8wlS3RYIbM,003w8wlS3RYIbM
album_name,,ZEIT,Shadows and Nightmares,Shadows and Nightmares,Shadows and Nightmares
artist_mid,002Vj8o12l10eK,000txhAP23aJfb,002Vj8o12l10eK,002Vj8o12l10eK,002Vj8o12l10eK
singer_names,[Two Steps From Hell],[Siriusmo],[Two Steps From Hell],[Two Steps From Hell],[Two Steps From Hell]
comment_count,1,0,1,0,0
singer_mids,[002Vj8o12l10eK],[000txhAP23aJfb],[002Vj8o12l10eK],[002Vj8o12l10eK],[002Vj8o12l10eK]
company,,Monkeytown Records,,,
release_date,None,2018-12-12,2006-09-16,2006-09-16,2006-09-16
